In [1]:
#importing libraries
import numpy as np
import tensorflow as tf
import helper

C:\Users\gsund\Anaconda3\envs\tensorflow\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
helper.download_dataset()
train_val_dataset, test_dataset = helper.load_images_labels()
train_val_images, train_val_labels = helper.extract_images_labels(train_val_dataset)
test_images, test_labels = helper.extract_images_labels(test_dataset)

from sklearn.model_selection import train_test_split
train_images, val_images, train_labels, val_labels = train_test_split(train_val_images, train_val_labels, test_size=0.2, random_state=42)

In [3]:
def reshape(x):
    return np.reshape(x, [-1, 32, 32, 3])

In [4]:
train_val_images = reshape(train_val_images)
test_images = reshape(test_images)
train_images = reshape(train_images)
val_images = reshape(val_images)

In [28]:
class CNN:
    
    def __init__(self, input_num, output_num, learning_rate, epochs, batch_size):
        
        height = input_num[0]
        width = input_num[1]
        depth = input_num[2]
        
        self.inputs = tf.placeholder(shape=[None, height, width, depth], dtype=tf.float32)
        self.outputs = tf.placeholder(shape=[None, output_num], dtype=tf.float32)
        
        self.learning_rate = learning_rate
        self.epochs = epochs
        self.batch_size = batch_size
        self.keep_prob = tf.placeholder(tf.float32)
        
        weights, bias = self.initialize_weights()
        self.weights = weights
        self.bias = bias
        
    def initialize_weights(self):
        
        weights_conv1 = tf.Variable(tf.truncated_normal(shape=[2, 2, 3, 32], mean=0.0, stddev=0.01, dtype=tf.float32))
        bias_conv1 = tf.Variable(tf.zeros(shape=[32]))
        
        weights_conv2 = tf.Variable(tf.truncated_normal(shape=[2, 2, 32, 64], mean=0.0, stddev=0.01, dtype=tf.float32))
        bias_conv2 = tf.Variable(tf.zeros(shape=[64]))
        
        weights_conv3 = tf.Variable(tf.truncated_normal(shape=[2, 2, 64, 128], mean=0.0, stddev=0.01, dtype=tf.float32))
        bias_conv3 = tf.Variable(tf.zeros(shape=[128]))
        
        weights_fc1 = tf.Variable(tf.truncated_normal(shape=[4*4*128, 512], mean=0.0, stddev=0.01, dtype=tf.float32))
        bias_fc1 = tf.Variable(tf.zeros(shape=[512]))
        
        weights_fc2 = tf.Variable(tf.truncated_normal(shape=[512, 128], mean=0.0, stddev=0.01, dtype=tf.float32))
        bias_fc2 = tf.Variable(tf.zeros(shape=[128]))
        
        weights_output_layer = tf.Variable(tf.truncated_normal(shape=[128, 10], mean=0.0, stddev=0.01, dtype=tf.float32))
        bias_output = tf.Variable(tf.zeros(shape=[10]))
        
        weights = {
            'conv1' : weights_conv1,
            'conv2' : weights_conv2,
            'conv3' : weights_conv3,
            'fully1' : weights_fc1,
            'fully2' : weights_fc2,
            'output' : weights_output_layer
        }
        
        bias = {
            'conv1' : bias_conv1,
            'conv2' : bias_conv2,
            'conv3' : bias_conv3,
            'fully1' : bias_fc1,
            'fully2' : bias_fc2,
            'output' : bias_output
        }
        
        return weights, bias
    
    def construct_network(self):
        
        conv_layer1 = self.conv2D(self.inputs, self.weights['conv1'], self.bias['conv1'])
        conv_layer1 = self.maxpool2D(conv_layer1)
        
        conv_layer2 = self.conv2D(conv_layer1, self.weights['conv2'], self.bias['conv2'])
        conv_layer2 = self.maxpool2D(conv_layer2)
        
        conv_layer3 = self.conv2D(conv_layer2, self.weights['conv3'], self.bias['conv3'])
        conv_layer3 = self.maxpool2D(conv_layer3)
        
        conv_layer = tf.reshape(conv_layer3, [-1, 4*4*128])
        
        """
        formula for calculating the new dimensions wrt convolution
        When Padding is 'SAME'
            new_height = ceil( float(input_height) / float(strides[0]) )
            new_width = ceil( float(input_width) / float(strides[1]) )
        When Padding is 'VALID'
            new_height = ceil( float(input_height - filter_height + 1) / float(strides[0]) )
            new_width = ceil( float(input_width - filter_width + 1) / float(strides[1]) )
            
        formula for calculating the new dimensions wrt max pooling
            new_height = ( (input_height - filter_height) / stride[0] ) + 1
            new_width = ( (input_width - filter_width) / stride[0] ) + 1
        """
        
        fully_connected_layer1 = tf.add(tf.matmul(conv_layer, self.weights['fully1']), self.bias['fully1'])
        fully_connected_layer1 = tf.nn.relu(fully_connected_layer1)
        fully_connected_layer1 = tf.nn.dropout(fully_connected_layer1, self.keep_prob)
        
        fully_connected_layer2 = tf.add(tf.matmul(fully_connected_layer1, self.weights['fully2']), self.bias['fully2'])
        fully_connected_layer2 = tf.nn.relu(fully_connected_layer2)
        fully_connected_layer2 = tf.nn.dropout(fully_connected_layer2, self.keep_prob)
        
        output_layer = tf.add(tf.matmul(fully_connected_layer2, self.weights['output']), self.bias['output'])
        #logits = tf.nn.softmax(output_layer)
        
        return output_layer
    
    def conv2D(self, inputs, weights, bias, conv_strides=1, conv_padding='SAME'):
        conv = tf.nn.conv2d(inputs, weights, strides=[1, conv_strides, conv_strides, 1], padding=conv_padding)
        conv = tf.nn.bias_add(conv, bias)
        return tf.nn.relu(conv)
    
    def maxpool2D(self, inputs, k=2, pool_strides=2, pool_padding='SAME'):
        # ksize refers to the filter
        # strides refer to the num of steps the filter should move
        return tf.nn.max_pool(inputs, ksize=[1, k, k, 1], strides=[1, pool_strides, pool_strides, 1], padding=pool_padding)
    
    def calculate_cost(self, outputs):
        
        loss = tf.nn.softmax_cross_entropy_with_logits_v2(logits=outputs, labels=self.outputs)
        cost = tf.reduce_mean(loss)
        return loss, cost
    
    def calculate_optimizer(self, loss):
        
        #return tf.train.GradientDescentOptimizer(learning_rate=self.learning_rate).minimize(loss)
        return tf.train.AdamOptimizer(0.01).minimize(loss)
    
    def calculate_accuracy(self, outputs):
        
        logits = tf.nn.softmax(outputs)
        correct_pred = tf.equal(tf.argmax(logits, 1), tf.argmax(self.outputs, 1))
        accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
        return accuracy
        
    def train(self, train_images, train_labels, validation_images, validation_labels, dropout=0.2):
        
        num_records = train_images.shape[0]
        init = tf.global_variables_initializer()

        with tf.Session() as sess:
            
            sess.run(init)
            
            for epoch in range(self.epochs):
                for ii in range(0, num_records, self.batch_size):

                    batch_train_images = train_images[ii: ii + self.batch_size]
                    batch_train_labels = train_labels[ii: ii + self.batch_size]

                    # Neural network
                    outputs = self.construct_network()
                    # cost function
                    loss, cost = self.calculate_cost(outputs)
                    # optimization function
                    optimizer = self.calculate_optimizer(loss)
                    # calculate accuracy
                    accuracy = self.calculate_accuracy(outputs)

                    sess.run(optimizer, feed_dict={
                        self.inputs : batch_train_images,
                        self.outputs : batch_train_labels,
                        self.keep_prob : 0.5
                    })

                    loss = sess.run(cost, feed_dict={
                        self.inputs : batch_train_images,
                        self.outputs : batch_train_labels,
                        self.keep_prob : 1.0
                    })

                    validation_accuracy = sess.run(accuracy, feed_dict={
                        self.inputs : validation_images, 
                        self.outputs : validation_labels,
                        self.keep_prob : 1.0
                    })

                    print("epoch {0:<3} Loss {1:0.3f} Accuracy {2:0.3f}".format(epoch, loss, validation_accuracy))
                        

    def test(): 
        pass

In [29]:
network = CNN(
    input_num = [32, 32, 3], 
    output_num = 10, 
    learning_rate = 0.001, 
    epochs = 100,
    batch_size = 5000)

In [ ]:
network.train(train_images, train_labels, val_images, val_labels, dropout=0.7)

In [ ]:
x = [21.0, 43.0, 55.0, 32.0, 88.0]
y = tf.placeholder(shape=(5,), dtype=tf.float16)

In [49]:
z = tf.nn.softmax(y)
p = tf.nn.softmax(z)

In [51]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    xy = sess.run(p, feed_dict={
        y: x
    })
    print(xy)

[0.1488 0.1488 0.1488 0.1488 0.4045]
